<a href="https://colab.research.google.com/github/Benjohn2001/The-Office-Predictor/blob/main/The_Office_Predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
import numpy as np
import pandas as pd

#Initial importing of data
parentURL = 'https://raw.githubusercontent.com/Benjohn2001/The-Office-Predictor/main/parent_reply.csv'
talkingURL = 'https://raw.githubusercontent.com/Benjohn2001/The-Office-Predictor/main/talking_head.csv'
parentDS = pd.read_csv(parentURL)
talkingDS = pd.read_csv(talkingURL)

#Drop the columns that are not needed
#Rename reply to quote so both datasets have matching column names
#Combine both the datasets and use reset_index with drop true to reset indexes
#and discard the old indexes, all data is now combined
#We have 27899 quotes when combined
talkingDS=talkingDS.drop(columns=['quote_id'])
parentDS=parentDS.drop(columns=["parent_id", "parent"])
parentDS=parentDS.rename(columns={'reply': 'quote'})
combinedDS = pd.concat([parentDS, talkingDS]).reset_index(drop=True)

#Remove quotes of length less than 5, removing 10575 quotes, now 17324 quotes 
#Specification has limit of 10000 samples so now random sample to 10000
i=0
for item in combinedDS['quote'].values:
    if(len(item.split())<5):
        combinedDS=combinedDS.drop(i,axis=0)
    i=i+1
sampled = combinedDS.sample(n=10000).reset_index(drop=True)

#Split dataset in 60% training, 20% validation, and 20% test
trainDS=sampled[:6000]
validationDS=sampled[6000:8000]
testDS=sampled[8000:]


In [31]:
sampledCounts=sampled['character'].value_counts()
trainCounts=trainDS['character'].value_counts()
validationCounts=validationDS['character'].value_counts()
testCounts=testDS['character'].value_counts()

tableDataCount={
    'Dataset':['Initial', 'Train', 'Validation', 'Test'],
    'Michael':[sampledCounts['Michael'],trainCounts['Michael'],validationCounts['Michael'],testCounts['Michael']],
    'Dwight':[sampledCounts['Dwight'],trainCounts['Dwight'],validationCounts['Dwight'],testCounts['Dwight']],
    'Jim':[sampledCounts['Jim'],trainCounts['Jim'],validationCounts['Jim'],testCounts['Jim']],
    'Pam':[sampledCounts['Pam'],trainCounts['Pam'],validationCounts['Pam'],testCounts['Pam']],
}

tableDataPercent={
    'Dataset':['Initial', 'Train', 'Validation', 'Test'],
    'Michael':[str(round(sampledCounts['Michael']/10000*100,2)),str(round(trainCounts['Michael']/6000*100,2)),str(round(validationCounts['Michael']/2000*100,2)),str(round(testCounts['Michael']/2000*100,2))],
    'Dwight':[str(round(sampledCounts['Dwight']/10000*100,2)),str(round(trainCounts['Dwight']/6000*100,2)),str(round(validationCounts['Dwight']/2000*100,2)),str(round(testCounts['Dwight']/2000*100,2))],
    'Jim':[str(round(sampledCounts['Jim']/10000*100,2)),str(round(trainCounts['Jim']/6000*100,2)),str(round(validationCounts['Jim']/2000*100,2)),str(round(testCounts['Jim']/2000*100,2))],
    'Pam':[str(round(sampledCounts['Pam']/10000*100,2)),str(round(trainCounts['Pam']/6000*100,2)),str(round(validationCounts['Pam']/2000*100,2)),str(round(testCounts['Pam']/2000*100,2))],
}

tableFrameCount=pd.DataFrame(tableDataCount)
print('Table displaying the amount of quotes for each character in each dataset\n')
print(tableFrameCount.to_string(index=False))

tableFramePercent=pd.DataFrame(tableDataPercent)
print('\nTable displaying the percentage of character quotes in each dataset\n')
print(tableFramePercent.to_string(index=False))

Table displaying the amount of quotes for each character in each dataset

   Dataset  Michael  Dwight  Jim  Pam
   Initial     4146    2396 1932 1526
     Train     2468    1470 1160  902
Validation      831     473  379  317
      Test      847     453  393  307

Table displaying the percentage of character quotes in each dataset

   Dataset Michael Dwight   Jim   Pam
   Initial   41.46  23.96 19.32 15.26
     Train   41.13   24.5 19.33 15.03
Validation   41.55  23.65 18.95 15.85
      Test   42.35  22.65 19.65 15.35
